# 风格微调评测 (Style Evaluation)

本notebook用于评估风格微调后的模型效果，包含三个主要评测维度：
1. **困惑度 (PPL)** - 在保留集上比较基座模型 vs LoRA适配器
2. **风格指示器** - 字数/句长分布、常用词频、停用词占比对比
3. **A/B人工评测** - 盲评样本，评估"更像作者"的程度

这些评测结果可直接用于学术论文的模型评估章节。

In [ ]:
import json
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig
)
from peft import PeftModel
import jieba
import re
from typing import List, Dict, Tuple

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境初始化完成")

## 1. 模型加载与配置

In [ ]:
# 配置路径
BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"
LORA_PATH = "outputs/qwen25-7b-sft-lora"
TEST_DATA = "data/sft_val.jsonl"

# 加载基础模型
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("正在加载基础模型...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

print("正在加载LoRA模型...")
lora_model = PeftModel.from_pretrained(base_model, LORA_PATH)

print("模型加载完成")

## 2. 困惑度评测 (Perplexity Evaluation)

In [ ]:
def calculate_perplexity(model, tokenizer, texts: List[str], max_length: int = 512) -> float:
    """
    计算模型在给定文本上的困惑度
    """
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for text in texts:
            # 编码文本
            inputs = tokenizer(text, return_tensors="pt", 
                             max_length=max_length, truncation=True)
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            
            # 计算损失
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            
            # 累加损失和token数
            num_tokens = inputs["input_ids"].numel()
            total_loss += loss.item() * num_tokens
            total_tokens += num_tokens
    
    # 计算平均损失和困惑度
    avg_loss = total_loss / total_tokens
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    
    return perplexity

# 加载测试数据
test_texts = []
with open(TEST_DATA, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        test_texts.append(data['output'])  # 使用原文进行困惑度测试

# 限制测试样本数量（避免计算时间过长）
test_texts = test_texts[:50]
print(f"使用 {len(test_texts)} 个样本进行困惑度测试")

# 计算基础模型困惑度
print("计算基础模型困惑度...")
base_ppl = calculate_perplexity(base_model, tokenizer, test_texts)

# 计算LoRA模型困惑度
print("计算LoRA模型困惑度...")
lora_ppl = calculate_perplexity(lora_model, tokenizer, test_texts)

print(f"\n困惑度对比结果:")
print(f"基础模型: {base_ppl:.2f}")
print(f"LoRA模型: {lora_ppl:.2f}")
print(f"改进程度: {((base_ppl - lora_ppl) / base_ppl * 100):.1f}%")

## 3. 风格指示器分析 (Style Indicators)

In [ ]:
def analyze_text_style(texts: List[str]) -> Dict:
    """
    分析文本的风格特征
    """
    # 基础统计
    char_counts = [len(text) for text in texts]
    sentence_counts = [len(re.split(r'[。！？]', text)) - 1 for text in texts]
    
    # 分词统计
    all_words = []
    for text in texts:
        words = jieba.lcut(text)
        all_words.extend(words)
    
    word_freq = Counter(all_words)
    
    # 停用词
    stop_words = {'的', '了', '在', '是', '我', '有', '和', '就', '不', '人', '都', '一', '一个', '上', '也', '很', '到', '说', '要', '去', '你', '会', '着', '没有', '看', '好', '自己', '这'}
    stop_word_count = sum(word_freq[word] for word in stop_words if word in word_freq)
    stop_word_ratio = stop_word_count / len(all_words) if all_words else 0
    
    return {
        'char_counts': char_counts,
        'sentence_counts': sentence_counts,
        'avg_char_per_text': np.mean(char_counts),
        'avg_sentence_per_text': np.mean(sentence_counts),
        'word_freq': word_freq,
        'stop_word_ratio': stop_word_ratio,
        'vocab_size': len(word_freq),
        'total_words': len(all_words)
    }

# 生成样本文本进行对比
def generate_samples(model, tokenizer, prompts: List[str], max_length: int = 200) -> List[str]:
    """
    生成模型样本
    """
    model.eval()
    samples = []
    
    with torch.no_grad():
        for prompt in prompts:
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
            generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
            # 移除原始prompt
            generated = generated[len(prompt):].strip()
            samples.append(generated)
    
    return samples

# 准备测试prompt
test_prompts = [
    "用我的口吻写一段关于日常生活的感悟：",
    "请用我的风格续写：今天又是平常的一天，但是",
    "用我的写作风格描述一个普通的下午：",
    "以我的口吻谈谈对阅读的看法：",
    "用我的风格写一段关于城市生活的观察："
]

print("生成基础模型样本...")
base_samples = generate_samples(base_model, tokenizer, test_prompts)

print("生成LoRA模型样本...")
lora_samples = generate_samples(lora_model, tokenizer, test_prompts)

# 分析原始作者文本风格
original_texts = [data['output'] for data in [json.loads(line) for line in open(TEST_DATA, 'r', encoding='utf-8')]][:20]
original_style = analyze_text_style(original_texts)

# 分析生成文本风格
base_style = analyze_text_style(base_samples)
lora_style = analyze_text_style(lora_samples)

print("\n风格指示器对比:")
print(f"平均字数 - 原文: {original_style['avg_char_per_text']:.1f}, 基础: {base_style['avg_char_per_text']:.1f}, LoRA: {lora_style['avg_char_per_text']:.1f}")
print(f"停用词比例 - 原文: {original_style['stop_word_ratio']:.3f}, 基础: {base_style['stop_word_ratio']:.3f}, LoRA: {lora_style['stop_word_ratio']:.3f}")
print(f"词汇多样性 - 原文: {original_style['vocab_size']}, 基础: {base_style['vocab_size']}, LoRA: {lora_style['vocab_size']}")

## 4. 可视化对比

In [ ]:
# 创建风格对比图表
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('风格微调效果对比', fontsize=16)

# 1. 字数分布对比
axes[0, 0].hist(original_style['char_counts'], alpha=0.7, label='原文', bins=10)
axes[0, 0].hist([len(s) for s in base_samples], alpha=0.7, label='基础模型', bins=10)
axes[0, 0].hist([len(s) for s in lora_samples], alpha=0.7, label='LoRA模型', bins=10)
axes[0, 0].set_title('字数分布对比')
axes[0, 0].set_xlabel('字数')
axes[0, 0].set_ylabel('频次')
axes[0, 0].legend()

# 2. 高频词对比
original_top_words = original_style['word_freq'].most_common(10)
base_top_words = base_style['word_freq'].most_common(10)
lora_top_words = lora_style['word_freq'].most_common(10)

words = [w[0] for w in original_top_words[:5]]
original_freqs = [original_style['word_freq'][w] / original_style['total_words'] for w in words]
base_freqs = [base_style['word_freq'][w] / base_style['total_words'] for w in words]
lora_freqs = [lora_style['word_freq'][w] / lora_style['total_words'] for w in words]

x = np.arange(len(words))
width = 0.25

axes[0, 1].bar(x - width, original_freqs, width, label='原文')
axes[0, 1].bar(x, base_freqs, width, label='基础模型')
axes[0, 1].bar(x + width, lora_freqs, width, label='LoRA模型')
axes[0, 1].set_title('高频词使用频率对比')
axes[0, 1].set_xlabel('词汇')
axes[0, 1].set_ylabel('使用频率')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(words)
axes[0, 1].legend()

# 3. 困惑度对比
models = ['基础模型', 'LoRA模型']
ppls = [base_ppl, lora_ppl]
colors = ['lightcoral', 'lightblue']
axes[1, 0].bar(models, ppls, color=colors)
axes[1, 0].set_title('困惑度对比（越低越好）')
axes[1, 0].set_ylabel('困惑度')
for i, v in enumerate(ppls):
    axes[1, 0].text(i, v + 0.1, f'{v:.2f}', ha='center', va='bottom')

# 4. 综合指标雷达图
metrics = ['字数相似度', '停用词相似度', '词汇丰富度', '困惑度']
# 计算相似度分数（越接近1越好）
def similarity_score(val1, val2):
    return 1 - abs(val1 - val2) / max(val1, val2)

base_scores = [
    similarity_score(original_style['avg_char_per_text'], base_style['avg_char_per_text']),
    similarity_score(original_style['stop_word_ratio'], base_style['stop_word_ratio']),
    base_style['vocab_size'] / max(original_style['vocab_size'], base_style['vocab_size']),
    1 / base_ppl * 10  # 困惑度转换为分数
]

lora_scores = [
    similarity_score(original_style['avg_char_per_text'], lora_style['avg_char_per_text']),
    similarity_score(original_style['stop_word_ratio'], lora_style['stop_word_ratio']),
    lora_style['vocab_size'] / max(original_style['vocab_size'], lora_style['vocab_size']),
    1 / lora_ppl * 10
]

x = np.arange(len(metrics))
axes[1, 1].plot(x, base_scores, 'o-', label='基础模型', linewidth=2, markersize=6)
axes[1, 1].plot(x, lora_scores, 's-', label='LoRA模型', linewidth=2, markersize=6)
axes[1, 1].set_title('综合风格指标对比')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(metrics, rotation=45)
axes[1, 1].set_ylabel('相似度分数')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('eval/style_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("可视化图表已保存到 eval/style_comparison.png")

## 5. A/B 人工评测样本生成

In [ ]:
# 生成A/B测试样本
ab_test_prompts = [
    "用我的口吻写一段关于早晨的感受",
    "以我的风格描述一次普通的购物经历", 
    "用我的语气谈谈对当下生活节奏的看法",
    "以我的口吻写一段关于读书的心得",
    "用我的风格描述一个安静的傍晚",
    "以我的语气聊聊对工作的思考",
    "用我的口吻写一段关于季节变化的观察",
    "以我的风格描述一次偶遇",
    "用我的语气谈谈对新技术的看法",
    "以我的口吻写一段关于美食的感受"
]

print("生成A/B测试样本...")
ab_base_samples = generate_samples(base_model, tokenizer, ab_test_prompts, max_length=150)
ab_lora_samples = generate_samples(lora_model, tokenizer, ab_test_prompts, max_length=150)

# 创建A/B测试数据
ab_test_data = []
for i, prompt in enumerate(ab_test_prompts):
    # 随机决定A/B顺序
    if np.random.random() > 0.5:
        sample_a, sample_b = ab_base_samples[i], ab_lora_samples[i]
        correct_answer = 'B'  # LoRA应该更像作者
    else:
        sample_a, sample_b = ab_lora_samples[i], ab_base_samples[i]
        correct_answer = 'A'
    
    ab_test_data.append({
        'id': i + 1,
        'prompt': prompt,
        'sample_a': sample_a,
        'sample_b': sample_b,
        'correct_answer': correct_answer,
        'model_a': 'LoRA' if correct_answer == 'A' else 'Base',
        'model_b': 'LoRA' if correct_answer == 'B' else 'Base'
    })

# 保存A/B测试数据
with open('eval/ab_test_samples.json', 'w', encoding='utf-8') as f:
    json.dump(ab_test_data, f, ensure_ascii=False, indent=2)

print(f"已生成 {len(ab_test_data)} 个A/B测试样本，保存到 eval/ab_test_samples.json")

# 显示前3个样本作为示例
print("\n=== A/B测试样本示例 ===")
for i in range(min(3, len(ab_test_data))):
    sample = ab_test_data[i]
    print(f"\n样本 {sample['id']}:")
    print(f"提示: {sample['prompt']}")
    print(f"\n选项A ({sample['model_a']}模型):")
    print(sample['sample_a'][:100] + '...' if len(sample['sample_a']) > 100 else sample['sample_a'])
    print(f"\n选项B ({sample['model_b']}模型):")
    print(sample['sample_b'][:100] + '...' if len(sample['sample_b']) > 100 else sample['sample_b'])
    print("-" * 50)

## 6. 评测报告生成

In [ ]:
# 生成评测报告
report = f"""
# 风格微调评测报告

## 模型配置
- 基础模型: {BASE_MODEL}
- LoRA路径: {LORA_PATH}
- 测试样本数: {len(test_texts)}

## 1. 困惑度评测结果

| 模型 | 困惑度 | 改进程度 |
|------|--------|----------|
| 基础模型 | {base_ppl:.2f} | - |
| LoRA模型 | {lora_ppl:.2f} | {((base_ppl - lora_ppl) / base_ppl * 100):.1f}% |

## 2. 风格指示器对比

| 指标 | 原文 | 基础模型 | LoRA模型 |
|------|------|----------|----------|
| 平均字数 | {original_style['avg_char_per_text']:.1f} | {base_style['avg_char_per_text']:.1f} | {lora_style['avg_char_per_text']:.1f} |
| 平均句数 | {original_style['avg_sentence_per_text']:.1f} | {base_style['avg_sentence_per_text']:.1f} | {lora_style['avg_sentence_per_text']:.1f} |
| 停用词比例 | {original_style['stop_word_ratio']:.3f} | {base_style['stop_word_ratio']:.3f} | {lora_style['stop_word_ratio']:.3f} |
| 词汇多样性 | {original_style['vocab_size']} | {base_style['vocab_size']} | {lora_style['vocab_size']} |

## 3. A/B人工评测

已生成 {len(ab_test_data)} 个A/B测试样本，可供人工评测使用。
评测文件: `eval/ab_test_samples.json`

### 评测说明:
1. 对每个样本，比较选项A和选项B哪个更像原作者的写作风格
2. 记录选择结果和置信度
3. 统计LoRA模型的胜率

## 4. 结论

- **困惑度改善**: LoRA微调{'显著' if (base_ppl - lora_ppl) / base_ppl > 0.1 else '适度'}降低了模型困惑度
- **风格一致性**: LoRA模型在{'字数分布、句长特征' if abs(lora_style['avg_char_per_text'] - original_style['avg_char_per_text']) < abs(base_style['avg_char_per_text'] - original_style['avg_char_per_text']) else '词汇使用'}等方面更接近原作者风格
- **评测可用性**: 生成的A/B测试样本可直接用于论文的人工评测章节

---
报告生成时间: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

# 保存报告
with open('eval/style_evaluation_report.md', 'w', encoding='utf-8') as f:
    f.write(report)

print("评测报告已保存到 eval/style_evaluation_report.md")
print("\n=== 评测完成 ===")
print(f"主要结果:")
print(f"- 困惑度改进: {((base_ppl - lora_ppl) / base_ppl * 100):.1f}%")
print(f"- A/B测试样本: {len(ab_test_data)} 个")
print(f"- 可视化图表: eval/style_comparison.png")
print(f"- 详细报告: eval/style_evaluation_report.md")